In [104]:
import numpy as np
import pandas as pd
import cv2 
import matplotlib.pyplot as plt


In [105]:
# img=cv2.imread("D:\imageprocessing\peakpx.jpg",cv2.IMREAD_GRAYSCALE)/255
# plt.imshow(img,cmap='gray')
# plt.show()


In [106]:
class conv:
    def __init__(self,num_filters,filter_size):
        self.num_filters=num_filters
        self.filters_size=filter_size
        self.conv_filter=np.random.randn(num_filters,filter_size,filter_size)/(filter_size*filter_size)
    def image_region(self,image):
        height,width=image.shape
        self.image=image
        for j in range(height-self.filters_size+1):
            for k in range(width-self.filters_size+1):
                image_patch=image[j:(j+self.filters_size),k:(k+self.filters_size)]
                yield image_patch,j,k
    def forward_prop(self,image):
        height, width=image.shape
        conv_out=np.zeros((height-self.filters_size+1,width-self.filters_size+1,self.num_filters))
        for image_patch,i,j in self.image_region(image):
            conv_out[i,j]= np.sum(image_patch*self.conv_filter,axis=(1,2))
        return conv_out
    def back_prop(self,dL_dout, learning_rate):
        dL_dF_params=np.zeros(self.conv_filter.shape)
        for image_patch,i,j in self.image_region(self.image):
            for k in range(self.num_filters):
                dL_dF_params[k]+=image_patch*dL_dout[i,j,k]
        self.conv_filter-=learning_rate*dL_dF_params
        return dL_dF_params
    






In [107]:
# conn=conv(18,7)
# out=conn.forward_prop(img)
# out.shape

In [108]:
# plt.imshow(out[:,:,3],cmap='gray')

In [109]:
class  max_pool:
    def __init__(self,filter_size):
        self.filter_size=filter_size
    def image_region(self,image):
        new_height=image.shape[0]//self.filter_size
        new_width=image.shape[1]//self.filter_size
        self.image=image
        for i in range(new_height):
            for j in range(new_width):
                image_patch=image[(i*self.filter_size):(i*self.filter_size+self.filter_size),(j*self.filter_size):(j*self.filter_size+self.filter_size)]
                yield image_patch,i,j
    def forward_prop(self,image):
        height,width,num_filters=image.shape
        output=np.zeros((height//self.filter_size,width//self.filter_size,num_filters))
        for image_patch,i,j in self.image_region(image):
            output[i,j]=np.amax(image_patch,axis=(0,1))
        return output
    def back_prop(self,dL_dout):
        dL_dmax_pool=np.zeros(self.image.shape)
        for image_patch,i,j in self.image_region(self.image):
            height,width,num_filters=image_patch.shape
            maximum_val=np.amax(image_patch,axis=(0,1))
            for il in range(height):
                for jl in range(width):
                    for kl in range(num_filters):
                        if image_patch[il,jl,kl]== maximum_val[kl]:
                            dL_dmax_pool[i*self.filter_size+il,j*self.filter_size+jl,kl]=dL_dout[i,j,kl]
        return dL_dmax_pool       




In [110]:
# conn2=max_pool(4)
# out2=conn2.forward_prop(out)
# out2.shape

In [111]:
# plt.imshow(out2[:,:,2],cmap='gray')

In [112]:
class softmax:
    def __init__(self,input_node,softmax_node):
        self.weight=np.random.randn(input_node,softmax_node)/input_node
        self.bias=np.zeros(softmax_node)
    def forward_prop(self,image):
        self.orig_im_shape=image.shape
        image_moodified=image.flatten()
        self.modified_input=image_moodified
        output_val=np.dot(image_moodified,self.weight)+self.bias
        self.out=output_val
        exp_out=np.exp(output_val)
        return exp_out/np.sum(exp_out,axis=0)
    def back_prop(self,dL_dout,learning_rate):
        for i, grad in enumerate(dL_dout):
            if grad==0:
                continue
            transformation_eq=np.exp(self.out)
            # print(transformation_eq)
            s_total=np.sum(transformation_eq)
            # gradient with respect to out(z)
            dy_dz=-transformation_eq[i]*transformation_eq/(s_total**2)
            dy_dz[i]=transformation_eq[i]*(s_total-transformation_eq[i])/(s_total**2)
            #gradients of totals against weights/biases/input
            dz_dw=self.modified_input
            dz_db=1
            dz_d_inp=self.weight
            #gradient of loss against totals

            dL_dz=grad*dy_dz
            # Gradients of loss against weights/biases/input
            dL_dw=dz_dw[np.newaxis].T@ dL_dz[np.newaxis]
            dL_db=dL_dz*dz_db
            dL_d_inp=dz_d_inp@dL_dz
            self.weight-=learning_rate*dL_dw
            self.bias-=learning_rate*dL_db
            return dL_d_inp.reshape(self.orig_im_shape)


In [113]:
# conn3=softmax(268*478*18,10)
# out3=conn3.forward_prop(out2)
# print(out3)

In [114]:
from keras.datasets import mnist
(X_train,y_train),(X_test,y_test)=mnist.load_data()

In [115]:
train_images=X_train[:1500]
train_labels=y_train[:1500]
test_images=X_test[:1500]
test_labels=y_test[:1500]


In [116]:
conv=conv(8,3)
pool=max_pool(2)
softmax=softmax(13*13*8,10)


In [117]:
def cnn_forward_prop(image,label):
    out_p=conv.forward_prop((image/255)-0.5)
    out_p=pool.forward_prop(out_p)
    out_p=softmax.forward_prop(out_p)
    cross_ent_loss=-np.log(out_p[label])
    accuracy_eval=1 if np.argmax(out_p)==label else 0
    return out_p,cross_ent_loss,accuracy_eval

In [118]:
def training_cnn(image,label,learn_rate=0.005):
    out,loss,acc=cnn_forward_prop(image,label)
    gradient=np.zeros(10)
    gradient[label]=-1/out[label]
    grad_back=softmax.back_prop(gradient,learn_rate)
    grad_back=pool.back_prop(grad_back)
    grad_back=conv.back_prop(grad_back,learn_rate)
    return loss,acc

In [119]:
for epoch1 in range(4):
    print('epochs %d---->' %(epoch1+1))
    #shuffle the training data
    shuffle_data=np.random.permutation(len(train_images))
    train_images=train_images[shuffle_data]
    train_labels=train_labels[shuffle_data]
    # training the cnn
    loss=0
    num_correct=0
    for i, (im,label) in enumerate(zip(train_images,train_labels)):
        if i %100==0:
            print('%d steps out of 100 steps: Average Loss %.3f and accuracy: %d%%'%(i+1,loss/100,num_correct))
            loss=0
            num_correct=0
        l1,accu=training_cnn(im,label)
        loss+=l1
        num_correct+=accu
    num_tests=len(test_images)
    print('test loss',loss/num_tests)
    print('test accuracy: ', num_correct/num_tests)

epochs 1---->
1 steps out of 100 steps: Average Loss 0.000 and accuracy: 0%


101 steps out of 100 steps: Average Loss 2.229 and accuracy: 20%
201 steps out of 100 steps: Average Loss 2.023 and accuracy: 39%
301 steps out of 100 steps: Average Loss 1.609 and accuracy: 45%
401 steps out of 100 steps: Average Loss 1.110 and accuracy: 65%
501 steps out of 100 steps: Average Loss 1.038 and accuracy: 65%
601 steps out of 100 steps: Average Loss 0.826 and accuracy: 71%
701 steps out of 100 steps: Average Loss 0.884 and accuracy: 71%
801 steps out of 100 steps: Average Loss 0.887 and accuracy: 70%
901 steps out of 100 steps: Average Loss 0.789 and accuracy: 70%
1001 steps out of 100 steps: Average Loss 0.495 and accuracy: 87%
1101 steps out of 100 steps: Average Loss 0.669 and accuracy: 82%
1201 steps out of 100 steps: Average Loss 0.728 and accuracy: 84%
1301 steps out of 100 steps: Average Loss 0.709 and accuracy: 80%
1401 steps out of 100 steps: Average Loss 0.574 and accuracy: 88%
test loss 0.045247404928781446
test accuracy:  0.051333333333333335
epochs 2---->
1 s

SyntaxError: invalid syntax (3921850951.py, line 1)